In [ ]:
import pickle
import json
import numpy as np
import pandas as pd
from keplergl import KeplerGl
import datetime
from shapely.geometry import shape
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Load data
with open('data/nyc-zip-code-tabulation-areas-polygons.geojson', 'r') as f:
    nyc_geojson = json.load(f)

with open('saved_files/reb_flows_mode2_1000cars_nyc_man_south.pkl', 'rb') as f:
    flow_data = pickle.load(f)

MANHATTAN_ZIP_CODES = ['10002', '10003', '10005', '10006', '10007', '10009', 
                       '10010', '10011', '10012', '10013', '10014', '10038']

In [ ]:
# Filter to Manhattan and extract centroids
manhattan_geojson = {
    'type': 'FeatureCollection',
    'features': [f for f in nyc_geojson['features'] 
                 if f['properties'].get('postalCode') in MANHATTAN_ZIP_CODES]
}

region_to_zip = {i: zc for i, zc in enumerate(MANHATTAN_ZIP_CODES)}
zip_to_region = {zc: i for i, zc in enumerate(MANHATTAN_ZIP_CODES)}

region_centroids = {}
for feature in manhattan_geojson['features']:
    postal = feature['properties']['postalCode']
    region_idx = zip_to_region[postal]
    geom = shape(feature['geometry'])
    region_centroids[region_idx] = (geom.centroid.x, geom.centroid.y)

In [ ]:
# Time settings
BASE_TIME = datetime.datetime(2013, 3, 8, 19, 0, 0)
INTERVAL_MINUTES = 3

def timestep_to_datetime(t):
    dt = BASE_TIME + datetime.timedelta(minutes=t * INTERVAL_MINUTES)
    return dt.strftime('%Y-%m-%d %H:%M:%S')

## Rebalancing Flows Visualization

In [ ]:
# Create rebalancing flow DataFrame
def create_flow_df(flow_data, agent_id, region_centroids, edges):
    flows = flow_data['agent_reb_flows'][agent_id]
    num_timesteps, num_edges = flows.shape
    
    records = []
    for t in range(num_timesteps):
        for edge_idx, (origin, dest) in enumerate(edges):
            flow_value = flows[t, edge_idx]
            origin_lon, origin_lat = region_centroids[origin]
            dest_lon, dest_lat = region_centroids[dest]
                
            records.append({
                'origin_lon': origin_lon,
                'origin_lat': origin_lat,
                'dest_lon': dest_lon,
                'dest_lat': dest_lat,
                'origin_region': origin,
                'dest_region': dest,
                'origin_zip': region_to_zip[origin],
                'dest_zip': region_to_zip[dest],
                'flow': float(flow_value),
                'timestep': t,
                'datetime': timestep_to_datetime(t),
                'agent': agent_id
            })
    
    return pd.DataFrame(records)

df_agent0 = create_flow_df(flow_data, 0, region_centroids, flow_data['edges'])

In [ ]:
# Kepler.gl configuration for rebalancing flows
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.71,
            'longitude': -73.99,
            'zoom': 12.5,
            'pitch': 55,
            'bearing': -50,
            'dragRotate': True
        },
        'visState': {
            'filters': [{
                'dataId': ['Agent 0 Rebalancing Flows'],
                'id': 'time-filter',
                'name': ['datetime'],
                'type': 'timeRange',
                'value': [df_agent0['datetime'].min(), df_agent0['datetime'].max()],
                'enlarged': True,
                'plotType': 'histogram',
                'animationWindow': 'free',
                'speed': 1,
                'yAxis': {'type': 'value', 'domain': [0, 179]}
            }],
            'layers': [
                {
                    'type': 'geojson',
                    'config': {
                        'dataId': 'Manhattan Zip Code Areas',
                        'label': 'Manhattan Regions',
                        'color': [96, 96, 96],
                        'columns': {'geojson': '_geojson'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.3,
                            'strokeOpacity': 0.8,
                            'thickness': 1,
                            'strokeColor': [0, 0, 0],
                            'stroked': True,
                            'filled': True,
                            'enable3d': False,
                            'wireframe': True
                        }
                    }
                },
                {
                    'type': 'arc',
                    'config': {
                        'dataId': 'Agent 0 Rebalancing Flows',
                        'label': 'Rebalancing Arcs',
                        'color': [0, 0, 255],
                        'columns': {
                            'lat0': 'origin_lat',
                            'lng0': 'origin_lon',
                            'lat1': 'dest_lat',
                            'lng1': 'dest_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Custom Blue to Yellow',
                                'type': 'sequential',
                                'category': 'Custom',
                                'colors': ['#0000FF', '#FFFF00']
                            },
                            'sizeRange': [0, 10],
                            'targetColor': [255, 255, 0]
                        },
                        'sizeField': {'name': 'flow', 'type': 'real'},
                        'sizeScale': 'linear'
                    },
                    'visualChannels': {
                        'sizeField': {'name': 'flow', 'type': 'real'},
                        'sizeScale': 'linear'
                    }
                }
            ]
        }
    }
}

manhattan_animated = KeplerGl(height=1000, config=config)
manhattan_animated.add_data(data=manhattan_geojson, name='Manhattan Zip Code Areas')
manhattan_animated.add_data(data=df_agent0, name='Agent 0 Rebalancing Flows')
manhattan_animated

In [ ]:
# Export rebalancing map
manhattan_animated.save_to_html(file_name='manhattan_rebalancing_flows.html')

## Pricing Policy Visualization

In [ ]:
# Create pricing DataFrames
def create_pricing_data(pricing_data, agent_id):
    num_timesteps = pricing_data[agent_id].shape[0]
    records = []
    
    for t in range(num_timesteps):
        for region_idx in range(len(MANHATTAN_ZIP_CODES)):
            price = pricing_data[agent_id][t, region_idx] * 2
            
            records.append({
                'region': region_idx,
                'zip_code': region_to_zip[region_idx],
                'price': float(price),
                'timestep': t,
                'datetime': timestep_to_datetime(t)
            })
    
    return pd.DataFrame(records)

df_pricing_agent0 = create_pricing_data(flow_data['agent_price_scalars'], 0)
df_pricing_agent1 = create_pricing_data(flow_data['agent_price_scalars'], 1)

# Price difference
price_diff = (flow_data['agent_price_scalars'][0] - flow_data['agent_price_scalars'][1]) * 2
df_pricing_diff = []
for t in range(price_diff.shape[0]):
    for region_idx in range(len(MANHATTAN_ZIP_CODES)):
        df_pricing_diff.append({
            'region': region_idx,
            'zip_code': region_to_zip[region_idx],
            'price_diff': float(price_diff[t, region_idx]),
            'timestep': t,
            'datetime': timestep_to_datetime(t)
        })
df_pricing_diff = pd.DataFrame(df_pricing_diff)

# Prepare GeoJSON for Plotly
geojson_for_plotly = manhattan_geojson.copy()
for feature in geojson_for_plotly['features']:
    feature['id'] = feature['properties']['postalCode']

In [ ]:
# Agent 0 Pricing Map
fig_agent0 = px.choropleth_mapbox(
    df_pricing_agent0,
    geojson=geojson_for_plotly,
    locations='zip_code',
    color='price',
    animation_frame='datetime',
    color_continuous_scale='RdBu_r',
    range_color=[df_pricing_agent0['price'].min(), df_pricing_agent0['price'].max()],
    mapbox_style='carto-darkmatter',
    zoom=11.5,
    center={'lat': 40.71, 'lon': -73.99},
    opacity=0.7,
    labels={'price': 'Price'},
    height=700
)

fig_agent0.update_layout(
    title={'text': '<b>Agent 0 Pricing Policy Over Time</b>', 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20}},
    coloraxis_colorbar=dict(title="Price", tickformat='.3f')
)

fig_agent0.show()

In [ ]:
# Agent 1 Pricing Map
fig_agent1 = px.choropleth_mapbox(
    df_pricing_agent1,
    geojson=geojson_for_plotly,
    locations='zip_code',
    color='price',
    animation_frame='datetime',
    color_continuous_scale='RdBu_r',
    range_color=[df_pricing_agent1['price'].min(), df_pricing_agent1['price'].max()],
    mapbox_style='carto-darkmatter',
    zoom=11.5,
    center={'lat': 40.71, 'lon': -73.99},
    opacity=0.7,
    labels={'price': 'Price'},
    height=700
)

fig_agent1.update_layout(
    title={'text': '<b>Agent 1 Pricing Policy Over Time</b>', 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20}},
    coloraxis_colorbar=dict(title="Price", tickformat='.3f')
)

fig_agent1.show()

In [ ]:
# Price Difference Map
max_abs_diff = max(abs(df_pricing_diff['price_diff'].min()), abs(df_pricing_diff['price_diff'].max()))

fig_diff = px.choropleth_mapbox(
    df_pricing_diff,
    geojson=geojson_for_plotly,
    locations='zip_code',
    color='price_diff',
    animation_frame='datetime',
    color_continuous_scale='RdBu_r',
    range_color=[-max_abs_diff, max_abs_diff],
    mapbox_style='carto-darkmatter',
    zoom=11.5,
    center={'lat': 40.71, 'lon': -73.99},
    opacity=0.7,
    labels={'price_diff': 'Price Difference<br>(Agent 0 - Agent 1)'},
    height=700
)

fig_diff.update_layout(
    title={'text': '<b>Price Difference: Agent 0 - Agent 1 Over Time</b>', 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20}},
    coloraxis_colorbar=dict(title="Price Diff<br>(A0 - A1)", tickformat='.3f')
)

fig_diff.show()

In [ ]:
# Export pricing maps
fig_agent0.write_html('manhattan_pricing_agent0.html')
fig_agent1.write_html('manhattan_pricing_agent1.html')
fig_diff.write_html('manhattan_pricing_difference.html')
print("✓ Exported all pricing maps")